In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv


In [ ]:
import csv
# 파일 읽어오기
f = open('../Losangeles_url_list.csv', 'r')
rdr = csv.reader(f)
co =  1
# 텍스트에 있는 URL list에 첨가하는 작업
URLLIST = list()
for line in rdr:
    for i in line:
        co = co +1
        URLLIST.append(i)
print(co)
f.close()

# 가게 시간정보 모델링 데이터

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')
dayday = list()
open_day = dict()
for URL in URLLIST[:10]:
    restaurant = dict()
    driver = webdriver.Chrome('../chromedriver', options=options)
    driver.get(URL)
    div_e1gr019l6 =driver.find_element_by_class_name('e1gr019l6')

    title = URL.split('/')[-1]
    restaurant['url'] = URL
    
    
    ################################배달 시간
    try:
        what_delivery=div_e1gr019l6.find_element_by_tag_name('h3').get_attribute('innerHTML').replace('<span>','').replace('</span>','')
        restaurant[what_delivery] = div_e1gr019l6.find_element_by_tag_name('div').get_attribute('innerHTML').replace('<span>','').replace('</span>','')
    except:
        print('배달시간 데이터 없는건가?')
    #################################배달 시간


    
    open_hour = dict()
    #################################식당 오픈시간
    div_efoongq0=driver.find_elements_by_class_name('efoongq0')

    for i in div_efoongq0:
        for a in i.find_elements_by_tag_name('div'):
            try:
                if int == type(int(a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')[0])):
                    open_hour[what_hour] =a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')

            except:
                what_hour = a.get_attribute('innerHTML').replace('<span>','').replace('</span>','')
            
        dayday.append(restaurant)
    restaurant['hour'] = open_hour
    ##################################
    open_day[title] = restaurant

In [ ]:
open_day

# store 모델 데이터


In [ ]:
# open_day

# 여기에 다 있습니다!!!

In [ ]:

for restaurant_data in open_day:
#     restaurant_name = restaurant_data
    URL = open_day[restaurant_data]['url']
    print(URL)
    res = requests.post(URL + '?client=customer.web&version=3.0.0')
    data=  res.json()

    open_day[restaurant_data]['name'] = data['data']['place']['name']
    try:
        open_day[restaurant_data]['description'] = data['data']['place']['brand']['description']
    except KeyError:
        open_day[restaurant_data]['description'] =''
#         print('설명이 없거나 brand가 없거나')
    

    ########################is_delivery 계산해야함 여기서!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    open_day[restaurant_data]['is_delivery'] = data['data']['place']['hours']['state']

    ######################## is_pickup
    try:
        open_day[restaurant_data]['is_pickup'] = data['data']['place']['is_pickup_enabled']
    except KeyError:
        open_day[restaurant_data]['is_pickup'] = False
#         print('배달이 안되는 건가 빈건가?')
    ########################delivery_price
    try:
        open_day[restaurant_data]['delivery_fee'] = float(data['data']['place']['delivery_fee_badge'].split(' ')[0][1::])
    except:
        open_day[restaurant_data]['delivery_fee'] = 0
#         print('Free Delivery  $0')

    ########################estimated_prep_time(만드는데 걸리는 시간)
    open_day[restaurant_data]['estimated_prep_time'] = data['data']['place']['estimated_prep_time']

    #########################
    try:
        open_day[restaurant_data]['is_partner'] =data['data']['place']['is_partner']
    except KeyError:
        open_day[restaurant_data]['is_partner'] = False
#         print('파트너가 아니구나')

    ########################thumbnail image(둘중 하나 쓰기)
    open_day[restaurant_data]['store_img'] = data['data']['place']['header_img']['original_url']
#     print(data['data']['place']['header_img']['resolutions'][0]['url'])

    ########################state?????

    ########################address
    open_day[restaurant_data]['street_address_1'] = data['data']['place']['street_address_1']
    open_day[restaurant_data]['city'] = data['data']['place']['city']


    ######################## lat(위도)
    open_day[restaurant_data]['lat'] = data['data']['place']['lat']

    ######################## lng(경도)
    open_day[restaurant_data]['lng'] = data['data']['place']['lng']
    
    ######################## lng(경도)
    try:
        open_day[restaurant_data]['badges'] = data['data']['place']['badges']
    except:
        open_day[restaurant_data]['badges'] = ''
        

    ########################badge
    # selenium !!!! selenium !!!! selenium !!!! 

    ########################food type
    open_day[restaurant_data]['food_type'] = data['data']['place']['primary_place_category']['slug']
    
    food_item = dict()
    
    try:
        for category in data['data']['place']['catalog']['categories']:

            
            categori_food_list = dict()
            
            for product in category['products']:
                item_info = dict()
                item_info['name']  = product['name']
                
                try:
                    item_info['img_url'] = product['img']['resolutions'][0]['url']
                except KeyError:
#                     print('이미지가 없다')
                    item_info['img_url'] = ''
                
                try:
                    item_info['description'] = product['description']
                except KeyError:
#                     print('설명이 없는거다')            
                    item_info['description'] = ''
                
                item_info['base_price'] = product['base_price']
#                 print('item_info==============' ,item_info )
                categori_food_list[item_info['name']] = item_info
    
    
#             print('categori_food_list=============================>', categori_food_list )
                
#########################################################################################
                food_options = dict()
                options_list = list()
                try:
                    for options in product['option_groups']:
                        option_name = item_info['name']+ ' = option_name = '  +options['name'] 

                        
                        #################################
                        option_category = dict()
                        option_list = list()
                        for option in options['options']:
                            option_info = dict()

                            option_info['name'] = option['name']

                            
                            
                            option_info['option_price'] = option['price']
#                             option_info['select'] = False
                            option_list.append(option_info)
#                         print(option_list)
                        option_category[options['name']] = option_list
                        try:
                            option_category['is_required'] = options['single']
                        except:
                            option_category['is_required'] = False
#                         print('option_category \n \n',option_category)
#                         print('\n \n')
                        options_list.append(option_category)
#                     print(options_list)
                    item_info['options'] = options_list
#                     print(item_info)
                except:
                    print('옵션이 없습니다!!!')
                    item_info['options'] = ''

                categori_food_list[item_info['name']] = item_info
            food_item[category['name']] = categori_food_list
    except:
        print('카탈로그가 비어있는거 이거는 음식정보가 없어서 제거해야 하는데')
        food_item['categori'] =''
        continue
    open_day[restaurant_data]['menu'] = food_item
    



# 각 가게마다 카테고리별 메뉴들 (item)

In [ ]:
open_day

In [2]:
from stores.models import *

In [3]:
Store

stores.models.Store

In [ ]:
for key in open_day.keys():
#     print(open_day[key])

#     if open_day[key]['is_pickup'] == False:
#         is_pickup = False
#     elif open_day[key]['is_pickup'] == True:
#         is_pickup = True
#     else:
#         is_pickup = False


#     Store.objects.create(
#         name = open_day[key]['name'],
#         description = open_day[key]['description'],
#         pickup = open_day[key]['is_pickup'],
#         delivery_price = open_day[key]['is_delivery'],
#         estimated_prep_time = open_day[key]['estimated_prep_time'],
#         is_partner = open_day[key]['is_partner'],
#         store_img = open_day[key]['store_img'],
#         address = open_day[key]['street_address_1'],
#         lat = open_day[key]['lat'],
#         lng = float(open_day[key]['lng']),
        
#     )
    
#     print(open_day[key]['name'])
#     print(open_day[key]['description'])
#     print(open_day[key]['is_pickup'])
    print(open_day[key]['is_delivery'])
    print(open_day[key]['delivery_fee'])
    print(open_day[key]['estimated_prep_time'])
    print(open_day[key]['is_partner'])
    print(open_day[key]['store_img'])
    print(open_day[key]['street_address_1'])
    print(open_day[key]['lat'])
    print(float(open_day[key]['lng']))
#     print(open_day[key]['badges'])
    print(open_day[key]['food_type'])

In [ ]:
print(open_day[key]['delivery_fee'])

In [4]:
Store.objects.create(
    url = 'https://naver.com',
    name = '까폼',
    description = '태국음식입니다.',
    delivery_time = '3:00 ~ 12:00',
    is_delivery = False,
    is_pickup = True,
    delivery_fee = 200,
    estimated_prep_time = 20,
    is_partner = True,
    address = '경기도 광명시',
    lat = 11.0,
    lng = -110.11,
    city = 'New York',
    food_type = 'fastfood',
    
    
)

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "postmates-db.ceu9zljhzswq.ap-northeast-2.rds.amazonaws.com" (15.165.151.146) and accepting
	TCP/IP connections on port 5432?


NameError: name 'Stroe' is not defined

In [ ]:
food_item = dict()

for URL in URLLIST:
    print(URL)
    res = requests.post(URL + '?client=customer.web&version=3.0.0')
    data=  res.json()
    try:
        for category in data['data']['place']['catalog']['categories']:

            item_info = dict()

            for product in category['products']:
                item_info['name']  = product['name']
                print(product['name'])
                try:
                    print(product['img']['resolutions'][0]['url'])
                    item_info['img_url'] = product['img']['resolutions'][0]['url']
                except KeyError:
                    print('이미지가 없다')
                    item_info['img_url'] = ''
                try:
                    item_info['description'] = product['description']
                except KeyError:
                    print('설명이 없는거다')            
                    item_info['description'] = ''
                item_info['base_price'] = product['base_price']
            print('-----------------------------')
            food_item[category['name']] = item_info
    except:
        print('카탈로그가 비어있는거 이거는 음식정보가 없어서 제거해야 하는데')
        food_item['categori'] =''
        continue


In [ ]:
open_day

In [ ]:
f = open('san_fransisco.json' ,'w')
f.write(str(open_day))
f.close()

In [ ]:
import json
json_str = json.dumps(open_day)
data = json.loads(json_str)

In [ ]:
data

In [ ]:
res = requests.post('https://postmates.com/merchant/jack-in-the-box-los-angeles-516-n-beaudry-ave' + '?client=customer.web&version=3.0.0')
data=  res.json()
for category in data['data']['place']['catalog']['categories']:
#     print(category)
        for product in category['products']:
            print(product['name'])
            print(product['img']['resolutions'][0]['url'])


# 각 음심마다 토핑 옵션

In [ ]:
for food in data['data']['place']['catalog']['categories']:
#     print('food_type' , food['name'])
    
    for product in food['products']:
#         print('    food_name' , product['name'])
        
        for options in product['option_groups']:
            ###########Topping category
            print('          options_category   : ' ,options['name'])
            
            for option in options['options']:
                ############topping name & topping price
                print('                           ' ,option['name'] , '   :  ',option['price'])

        print('-----------------------------')
